In [ ]:
results.extend( client.get(data_uuid, offset=start, limit=chunk_size))

In [ ]:
import pandas as pd
from sodapy import Socrata
client = Socrata('data.cityofnewyork.us', 'ERDGUvZUDVHLJnGt3M7JwSJR3')
def fetch_data(data_uuid, nrows = None, usecols = []):
    record_length = int(client.get(data_uuid, select="COUNT(*)")[0]['COUNT'])
    nrows = min(0 if nrows is None else nrows, record_length)
    start = 0
    results =[]
    while start <= nrows:
      results.extend( client.get(data_uuid, offset=start, limit=5000))
      start = start + 5000
    return pd.DataFrame.from_records(results)
df1 = fetch_data('2gcj-p28u')
df2 = fetch_data('nyuc-6mit')
df = pd.merge(df1, df2, left_on=['community_school_district','school_level'], right_on=['community_school_district','school_level'])

In [ ]:
df2

,dbn,of_full_time_teachers_with,of_f_status_teachers_with,of_itinerant_teachers_with,sum_of_f_status_and_itinerant,total_of_part_time_and_full,location_code,community_school_district,city_council_district,school_level
0,Citywide:,152,3,0,3,155,NaN,NaN,NaN,NaN
1,01M015,0,0,0,0,0,M015,1,2,ELEMENTARY
2,01M019,0,0,0,0,0,M019,1,2,ELEMENTARY
3,01M020,0,0,0,0,0,M020,1,1,ELEMENTARY
4,01M034,0,0,0,0,0,M034,1,2,K-8
...,...,...,...,...,...,...,...,...,...,...
1577,75X596,0,0,0,0,0,X596,75,12,ELEMENTARY
1578,75X721,0,0,0,0,0,X721,75,13,HIGH SCHOOL
1579,75X723,0,0,0,0,0,X723,75,12,K-12 ALL GRADES
1580,75X754,0,0,0,0,0,X754,75,8,HIGH SCHOOL


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import re
import os
import shutil
from glob import glob
from tqdm import tqdm
from IPython.display import clear_output
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 100)

In [ ]:
metadata_df = pd.read_csv('/content/drive/MyDrive/metavis/socrata_nycopendata_metadata_fetched_20211105.csv')

In [ ]:
# metadata_df.metadata = metadata_df.metadata.apply(eval)
metadata_df['update_frequency'] = metadata_df['metadata'].apply(lambda x: x.get('custom_fields',{}).get('Update',{}).get('Update Frequency',np.nan))

In [ ]:
import subprocess
def wget_tabular_data(data_uuid, timeout_seconds = 5*60):
  print('\n'+data_uuid+'\n')
  command = "wget https://data.cityofnewyork.us/api/views/"+data_uuid+"/rows.csv?accessType=DOWNLOAD -O /content/drive/MyDrive/metavis/data/"+data_uuid+".csv"
  try:
    command_output = subprocess.check_output(command, shell=True, timeout=timeout_seconds)
  except:
    if os.path.exists("/content/drive/MyDrive/metavis/data/"+data_uuid+".csv"):
      os.remove("/content/drive/MyDrive/metavis/data/"+data_uuid+".csv")
  clear_output()

In [ ]:
# !pip install sodapy -q
# from sodapy import Socrata

# ## Reference: http://holowczak.com/getting-started-with-nyc-opendata-and-the-socrata-api/5/
# data_url='data.cityofnewyork.us'    # The Host Name for the API endpoint (the https:// part will be added automatically)
# app_token='ERDGUvZUDVHLJnGt3M7JwSJR3'   # The app token created in the prior steps
# client = Socrata(data_url, app_token)      # Create the client to point to the API endpoint
# client.timeout = 60 # Set the timeout to 60 seconds    

# def fetch_data(data_uuid):
#   # Get the total number of records in our target data set
#   record_count = client.get(data_uuid, select="COUNT(*)")
#   record_length = int(record_count[0]['COUNT'])
#   if record_length>10e6:
#     print('\n',data_uuid,'data length:',record_length)
#   start = 0             # Start at 0
#   chunk_size = 2000 if record_length <= 1e6 else 5000  # Fetch n rows at a time
#   results =[]           # Empty out our result list
#   while True:
#       # Fetch the set of records starting at 'start'
#       results.extend( client.get(data_uuid, offset=start, limit=chunk_size))
#       # Move up the starting record
#       start = start + chunk_size
#       # If we have fetched all of the records, bail out
#       if (start > record_length):
#           break
#   # Convert the list into a data frame
#   data = pd.DataFrame.from_records(results)
#   return data

In [ ]:
!pip install geojson -U -q

In [ ]:
import requests
import random

def request_url(url, verbose = True):

  headers_list = [{'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8', 'Referer': 'https://www.google.com/', 'DNT': '1', 'Connection': 'keep-alive', 'Upgrade-Insecure-Requests': '1'}, {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8', 'Referer': 'https://www.google.com/', 'DNT': '1', 'Connection': 'keep-alive', 'Upgrade-Insecure-Requests': '1'}, {'Connection': 'keep-alive', 'DNT': '1', 'Upgrade-Insecure-Requests': '1', 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9', 'Sec-Fetch-Site': 'none', 'Sec-Fetch-Mode': 'navigate', 'Sec-Fetch-Dest': 'document', 'Referer': 'https://www.google.com/'}, {'Connection': 'keep-alive', 'Upgrade-Insecure-Requests': '1', 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9', 'Sec-Fetch-Site': 'same-origin', 'Sec-Fetch-Mode': 'navigate', 'Sec-Fetch-User': '?1', 'Sec-Fetch-Dest': 'document', 'Referer': 'https://www.google.com/'}] # Reference: https://www.scrapehero.com/how-to-fake-and-rotate-user-agents-using-python-3

  try:
      headers = random.choice(headers_list)
      response = requests.get(url, headers = headers)
      response.raise_for_status() # Raise Exception when response was not successful
  except requests.exceptions.HTTPError as http_err:
      print('[Error] HTTP error occurred: '+str(http_err))
      return requests.models.Response() # Return empty response
  except Exception as err:
      print('[Error] Other error occurred: '+str(err))
      return requests.models.Response() # Return empty response
  else:
      if verbose:
          print('[Success] The website at "'+url+'" is collected successfully.')
      return response

# pip install geojson
from geojson import dump
def request_geo_data(data_uuid):
  print('\n'+data_uuid+'\n')
  r = request_url("https://data.cityofnewyork.us/api/geospatial/"+data_uuid+"?method=export&format=GeoJSON")
  # j = json.loads(r.content)
  j = r.json()
  with open("/content/drive/MyDrive/metavis/data/"+data_uuid+".geojson", "w") as f:
    dump(j, f)
    
# # pip install geopandas
# import geopandas as gpd
# g = gpd.read_file("/content/drive/MyDrive/metavis/data/"+data_uuid+".geojson")

In [ ]:
metadata_df = pd.read_csv('/content/drive/MyDrive/metavis/socrata_nycopendata_metadata_fetched_20211105.csv')
metadata_df['viewType'].value_counts()

tabular    2786
geo         241
blobby      190
href        188
Name: viewType, dtype: int64

In [ ]:
tabular_data_uuid_list = sorted(metadata_df.loc[metadata_df['viewType'] == 'tabular','data_uuid'].tolist())

# collected_data_uuid_list = [p.split('/')[-1].split('.')[0] for p in glob("/content/drive/MyDrive/metavis/data/*")]
# for data_uuid in tqdm(tabular_data_uuid_list):
#   if not data_uuid in collected_data_uuid_list:
#     wget_tabular_data(data_uuid)

In [ ]:
# geo_data_error_list = []
# geo_data_uuid_list = sorted(metadata_df.loc[metadata_df['viewType'] == 'geo','data_uuid'].tolist())
# collected_data_uuid_list = [p.split('/')[-1].split('.')[0] for p in glob("/content/drive/MyDrive/metavis/data/*")]

# for data_uuid in tqdm(geo_data_uuid_list):
#   if not data_uuid in collected_data_uuid_list + geo_data_error_list:
#     try:
#       request_geo_data(data_uuid)
#     except:
#       geo_data_error_list.append(data_uuid)
# geo_data_error_list # ['nqwf-w8eh'] # Building Footprints

In [ ]:
data_uuid_to_name_mapping = metadata_df.set_index('data_uuid')['name'].to_dict()

server_error_data_uuid_list = ['34hf-h2fw','3khw-qi8f','td5q-ry6d', 'tqkr-fgym']

collected_data_uuid_list = [p.split('/')[-1].split('.')[0] for p in glob("/content/drive/MyDrive/metavis/data/*")]
for data_uuid in tqdm(tabular_data_uuid_list):
  if not data_uuid in collected_data_uuid_list + server_error_data_uuid_list:
    data_name = data_uuid_to_name_mapping[data_uuid].lower()
    if 'trip data' in data_name or 'tripdata' in data_name:
      pass
    else:
      command = "wget https://data.cityofnewyork.us/api/views/"+data_uuid+"/rows.csv?accessType=DOWNLOAD -O /content/drive/MyDrive/metavis/data/"+data_uuid+".csv"
      print("!"+command+'\n')
      

In [ ]:
# pip install folderstats
import folderstats
data_stats = folderstats.folderstats("/content/drive/MyDrive/metavis/data/", ignore_hidden=True)
# pip install humanfriendly
from humanfriendly import format_size
data_stats['readable_size'] = data_stats['size'].apply(lambda x: format_size(x))